In [21]:
import Data.Manifold.TreeCover
import Data.Random
import Data.Random.Manifold
import Data.Manifold.Types
import Data.LinearMap.HerMetric
import Data.VectorSpace
import Data.Semigroup

From [dynamic-plot](http://hackage.haskell.org/package/dynamic-plot):

In [26]:
import Graphics.Dynamic.Plot.R2
viewRange = plot [forceXRange (-7,11), forceYRange (-2,4)]

In [32]:
tf <- runRVar (uncrtFuncIntervalSpls 100 (-2,5) $ \x -> (x*0 + 1, 1)) StdRandom
plotWindow [plot tf, viewRange]

GraphWindowSpecR2{lBound=-7.000000000000002, rBound=11.0, bBound=-2.0, tBound=4.0, xResolution=640, yResolution=480}

In [31]:
Option (Just tf') = filterDEqnSolution_static (\(Shade' (_,y) _) -> Shade' (denseLinear $ \δx -> y*δx) zeroV) tf
plotWindow [plot tf', viewRange]

<interactive>:1:1-112: Irrefutable pattern failed for pattern Option (Just tf')